In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import scipy.stats as st
from pprint import pprint
import time
from scipy.stats import linregress
import reverse_geocoder as rg

In [2]:
# Set base url parameters
base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
magnitude = "minmagnitude=5"
limit = "limit=19999"
year=1980

# set up lists to hold reponse info
time_ls = []
lat_ls = []
lng_ls = []
depth_ls = []
mag_ls = []
sig_ls = []
place_ls = []

# iterate through years
for x in range (0,10):
    time = f"starttime={year}-01-01&endtime={year}-12-31"
    query_url = (f"{base_url}&{time}&{magnitude}&{limit}")
    response = requests.get(query_url).json()

    # Append data to appropriate list
    for i in range (0, len(response["features"])):
        mag_ls.append(response["features"][i]["properties"]["mag"])
        place_ls.append(response["features"][i]["properties"]["place"])
        time_ls.append(response["features"][i]["properties"]["time"])
        sig_ls.append(response["features"][i]["properties"]["sig"])
        lat_ls.append(response["features"][i]["geometry"]["coordinates"][1])
        lng_ls.append(response["features"][i]["geometry"]["coordinates"][0])
        depth_ls.append(response["features"][i]["geometry"]["coordinates"][2])
    # counter for next year
    
    year=year+1

In [9]:
# create dictionary
data = {
    "Time": time_ls,
    "Lat": lat_ls,
    "Lng": lng_ls,
    "Depth": depth_ls,
    "Mag": mag_ls,
    "Sig": sig_ls,
    "Place": place_ls
}
# create dataframe
data_df = pd.DataFrame(data)

#convert timestamp to dates
data_df["Time"] = pd.to_datetime(data_df["Time"], unit="ms")
data_df["Month"] = pd.DatetimeIndex(data_df["Time"]).month
data_df["Day"] = pd.DatetimeIndex(data_df["Time"]).day
data_df["Year"] = pd.DatetimeIndex(data_df["Time"]).year

# Print dataframe

data_df= data_df[["Lat","Lng","Depth","Mag","Sig","Place","Time","Month","Day","Year"]]

data_df.head()

,Lat,Lng,Depth,Mag,Sig,Place,Time,Month,Day,Year
0,-17.575,-172.210,33.0,5.4,449,Tonga region,1980-12-30 18:53:41.700,12,30,1980
1,0.148,97.311,33.0,5.4,449,"Nias region, Indonesia",1980-12-30 15:48:38.900,12,30,1980
2,17.160,-97.892,74.0,5.2,416,"Oaxaca, Mexico",1980-12-29 22:10:21.700,12,29,1980
3,18.690,-62.832,33.0,5.1,400,"Anguilla region, Leeward Islands",1980-12-29 05:43:45.600,12,29,1980
4,-25.176,-70.024,54.0,5.0,385,"Antofagasta, Chile",1980-12-28 17:03:16.100,12,28,1980


In [10]:
data_df.to_csv("output_data/earthquake_data_1980-89.csv", index= False)